In [34]:
import json
import yfinance as yf

import openai
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

client = openai.Client()

def retorna_cotacao_acao_historica(
        ticker,
        periodo='1mo'
):
    ticker_obj = yf.Ticker(f'{ticker}.SA')
    # ticker_obj = yf.Ticker(f'{ticker}')
    hist = ticker_obj.history(period=periodo)['Close']
    hist.index = hist.index.strftime('%Y-%m-%d')
    hist = round(hist, 2)
    if len(hist) > 30:
        slice_size = int(len(hist) / 30)
        hist = hist.iloc[::-slice_size][::-1]
    return hist.to_json()


tools = [
    {
        'type': 'function',
        'function': {
            'name': 'retorna_cotacao_acao_historica',
            'description': 'Retorna a cotação diária histórica para uma ação da Bovespa',
            'parameters': {
                'type': 'object',
                'properties': {
                    'ticker': {
                        'type': 'string',
                        'description': 'O ticker da ação. Exemplo: "ABEV3" para Ambev, "PETR4" para Petrobras, etc.'
                    },
                    'periodo': {
                        'type': 'string',
                        'description': 'O período que será retornado de dados históricos \
                                        sendo "1mo" equivalente a um mês de dados, "1d" a \
                                        1 dia e "1y" a 1 ano',
                        'enum': ["1d","5d","1mo","6mo","1y","5y","10y","ytd","max"]
                    }
                }
            }
        }
    }

]

funcoes_disponiveis = {'retorna_cotacao_acao_historica': retorna_cotacao_acao_historica}

mensagens = [{'role': 'user', 'content': 'Qual a cotação da Suzano agora'}]

resposta = client.chat.completions.create(
    messages=mensagens,
    model='gpt-4o-mini',
    tools=tools,
    tool_choice='auto'
)


In [40]:
resposta.choices[0].message.tool_calls

[ChatCompletionMessageToolCall(id='call_AFJ46PJqmehYazq4dYQe6DRb', function=Function(arguments='{"ticker":"SUZB3","periodo":"1d"}', name='retorna_cotacao_acao_historica'), type='function')]

In [41]:
tool_calls = resposta.choices[0].message.tool_calls

if tool_calls:
    mensagens.append(resposta.choices[0].message)
    for tool_call in tool_calls:
        func_name = tool_call.function.name
        function_to_call = funcoes_disponiveis[func_name]
        func_args = json.loads(tool_call.function.arguments)
        func_return = function_to_call(**func_args)
        mensagens.append({
            'tool_call_id': tool_call.id,
            'role': 'tool',
            'name': func_name,
            'content': func_return
        })
    segunda_resposta = client.chat.completions.create(
        messages=mensagens,
        model='gpt-4o-mini',
    )
    mensagens.append(segunda_resposta.choices[0].message)

segunda_resposta.choices[0].message.content

'A cotação da ação da Suzano (SUZB3) está em R$ 55,70.'

In [36]:
segunda_resposta.choices[0].message.content

'A cotação da ação da Suzano (SUZB3) está em R$ 55,70.'

In [37]:
func_name = tool_call.function.name
function_to_call = funcoes_disponiveis[func_name]
func_args = json.loads(tool_call.function.arguments)
func_return = function_to_call(**func_args)


func_return

'{"2024-10-08":55.7}'

In [38]:
func_args

{'ticker': 'SUZB3', 'periodo': '1d'}

In [39]:
tool_call

ChatCompletionMessageToolCall(id='call_AFJ46PJqmehYazq4dYQe6DRb', function=Function(arguments='{"ticker":"SUZB3","periodo":"1d"}', name='retorna_cotacao_acao_historica'), type='function')